In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
import numpy as np
import pandas as pd
import time

from src.fitting import train_mf_native, reviews_dataset, evaluate_recommendations
from src.fitting.mf_native import preprocess, get_reviews_by_user, update_X, get_loss

In [3]:
reviews_train, reviews_validation, reviews_test, images = reviews_dataset()

In [ ]:
recommender = train_mf_native(
    train_data=reviews_train,
    images=images,
    d=20,
    alpha=.01,
    beta=.01,
    max_als_epochs=10,
    sgd_learning_rate=.001,
    max_sgd_epochs=100000,
    sgd_batch_size=10,
)

als_epoch=0
Computing X
199.72s
Computing Y
0.04s
Computing loss


In [5]:
print("Computing X")
start = time.time()
update_X(
    X=X,
    Y=Y,
    I=I,
    image_indices=image_indices,
    ratings=ratings,
    start_indices=start_indices,
    end_indices=end_indices,
    n=n,
    d=d,
    alpha=alpha,
)
end = time.time()
print(f"{end - start:.2f}s")

Computing X
181.77s


In [6]:
import random

Y = np.random.normal(size=(k, d))

learning_rate = .001
beta_1, beta_2 = .9, .999
sample_size = 10
eps = 1e-8
beta_1_pow = beta_1
beta_2_pow = beta_2

In [7]:
m = np.zeros((k, d))
v = np.zeros((k, d))

loss = 0.0

for epoch in range(1, 100000+1):
    if epoch % 10000 == 0:
        print(epoch)
        print(loss / (10000 * sample_size) + beta / (k * d) * (Y ** 2).sum())
        loss = 0.0
    
    g = np.zeros((k, d))
    
    sample = random.sample(range(n), sample_size)
    for u in sample:
        start_index = start_indices[u]
        end_index = end_indices[u]
        
        x_u = X[u]
        r_u = ratings[start_index : end_index]
        I_u = I[image_indices[start_index : end_index]]
        
        g -= 2/(end_index - start_index) * ((r_u - (I_u @ Y) @ x_u) @ I_u).reshape(-1, 1) @ x_u.reshape(1, -1)
        
        loss += ((r_u - (I_u @ Y) @ x_u) ** 2).sum() / (end_index - start_index)
    
    g = g / sample_size + 2 * beta / (k * d) * Y
    
    m = beta_1 * m + (1 - beta_1) * g
    v = beta_2 * v + (1 - beta_2) * (g ** 2)
    
    update = learning_rate * (m / (1. - beta_1_pow)) / (np.sqrt(v  / (1. - beta_2_pow)) + eps)  # ADAM
    # update = learning_rate * g  # SGD
    Y -= update
    
    beta_1_pow *= beta_1
    beta_2_pow *= beta_2

10000
4.794757785259492
20000
1.3619237607658938
30000
0.7192399222513858
40000
0.4351773335456885
50000
0.3034218034629517
60000
0.22348204564111568
70000
0.17385626417662686
80000
0.14403376756660274
90000
0.1220951673030781
100000
0.10670580548837964
